# **Welcome to Data Project 1. This is an assignment for DS 2002: Data Science Systems**
# This project has been completed by JP Meyer (kxn4mn) and Lily Rademacher (amn8ge)

Start by downloading the following data sets as either a .csv or .geojson files by going to these links:

Charlottesville Arrests: https://opendata.charlottesville.org/datasets/d558ab0e09fe4f509280bedf6f8793ed_22/explore

Historical Fire Data: https://opendata.charlottesville.org/datasets/31093f4678b44cc38e0e8347b2336e44_32/explore

**Ensure that these files have been imported into your Colab notebook file section before continuing!**
- Before importing the .geojson files, rename them Arrests.json and Historical_Fire_Data.json. The code will not function correctly unless the file type has been changed to .json from .geojson

Import the necessary libraries.

In [ ]:
import pandas as pd
import csv
import json

!pip install SQLAlchemy
from sqlalchemy import create_engine

Based on the user's input, the following code block downloads and reads the chosen file, imports the data into a pandas data frame, then reports the initial number of columns and records in that data frame.

In [ ]:
import_type = input("What type of file are you importing? (csv, json, sql): ")

if import_type == "csv":
    # gets file name from user, then converts csv to data frame
    file_name = input("Enter the name of the CSV file. Do not include the file type (e.g. '.csv'): ")
    df = pd.read_csv(file_name + ".csv")
elif import_type == "json":
    file_name = input("Enter the name of the JSON file: ")
    try:
        # both arrests and fire are originally .geojson files and have different
        # formatting than .json files, so need to extract data and convert it
        # to the same dictionary format as .json files have before being able to
        # read and write to pandas df
        with open(file_name + ".json", 'r', encoding='utf-8') as f:
            data = json.load(f)
        features = data['features']
        properties_list = [feature['properties'] for feature in features]
        df = pd.DataFrame(properties_list)
    except ValueError:
        print("Error: Could not import with 'records' orient. Check JSON structure or try different orient options.")
elif import_type == "sql":
    # reads sql database file and converts to data frame
    db_name = input("Enter the name of the database: ")
    table_name = input("Enter the name of the table: ")
    df = pd.read_sql_table(table_name, db_name)

# printing summary of data ingestion
num_cols = len(df.columns)
num_records = len(df)

print("\n" + "Your dataset, " + file_name + ", has been imported.")
print(file_name, "contains", num_cols, "columns and", num_records, "records.")


What type of file are you importing? (csv, json, sql): csv
Enter the name of the CSV file. Do not include the file type (e.g. '.csv'): Historical_Fire_Data

Your dataset, Historical_Fire_Data, has been imported.
Historical_Fire_Data contains 18 columns and 2726 records.


The following code block modifies the original files:
- For the Charlottesville Arrests data, it removes the Statute and NameSuffix columns and filters out all entries where the arrestee is a woman.
- For the Historical Fire data, it removes the latitude and longitude columns and filters entries so that only incidents lasting longer than 10 minutes are included.

After these transformations, a post-processing assessment of the updated number of columns and records are printed.

The first five entries in the data frame are also printed.

In [ ]:
if file_name == "Arrests":
    # drop irrelevant cols, filter out women, update # of cols/recs
    df = df.drop(columns=['Statute', 'NameSuffix'])
    df = df[df['Sex'] != 'Female']

    # printing data processing summary
    new_num_cols = len(df.columns)
    new_num_records = len(df)
    print("After processing,", file_name, "now contains", new_num_cols, "columns and", new_num_records, "records.")
    print("\n",df.head())
elif file_name == "Historical_Fire_Data":
    # drop irrelevant cols, filter out short incidents, update # of cols/recs
    df = df.drop(columns=['Latitude', 'Longitude'])
    df = df[df['IncidentDurationMinutes'] > 10]

    # printing data processing summary
    new_num_cols = len(df.columns)
    new_num_records = len(df)
    print("After processing,", file_name, "now contains", new_num_cols, "columns and", new_num_records, "records.")
    print("\n",df.head())
else:
    # protecting for errors
    print("Error: Incorrect Dataset Imported")

After processing, Historical_Fire_Data now contains 16 columns and 2480 records.

   AddressStandardized  AlarmDateTimeAmericaNY            CauseOfIgnition  \
1       1014 RIDGE ST  2023/06/28 00:55:59+00              Unintentional   
2        38 SPRING CT  2023/06/27 10:43:22+00                        NaN   
3   1819 RIVER INN LN  2023/06/22 16:24:04+00                        NaN   
4     637 OAK HILL CT  2023/06/17 05:17:59+00                        NaN   
5     1404 MIDLAND ST  2023/06/14 19:04:02+00  Cause under investigation   

                           FireAreaOfOrigin  \
1        Laundry area, wash house (laundry)   
2                                       NaN   
3                                       NaN   
4                                       NaN   
5  Highway, parking lot, street: on or near   

                                FireType  \
1                          Building fire   
2                          Building fire   
3    Cooking fire, confined to container   
4

The following code takes the user's chosen
conversion type and converts the previously created data frame to the chosen format. Then, the new file is stored locally in the Colab notebook file section.

In [ ]:
convert_type = input(f"What would you like to convert {file_name}.{import_type} to? (csv, json, sql): ")

if convert_type == import_type:
    # protecting for errors
    print("\nError: You cannot convert a file to its current format. Please choose a different format.")
elif convert_type != import_type and convert_type == "csv":
    # using the data frame to csv function
    df.to_csv(file_name + "_converted.csv", index=False)
    print(f"\nFile converted successfully to CSV as {file_name}_converted.csv!")
elif convert_type != import_type and convert_type == "json":
    # using the data frame to json function
    df.to_json(file_name + "_converted.json", orient="records")
    print(f"\nFile converted successfully to JSON as {file_name}_converted.json!")
elif convert_type != import_type and convert_type == "sql":
    # converting df to SQL w/o querying SQL database or using a server by using
    # in memory sqlite database and engine
    engine = create_engine('sqlite:///' + file_name + '_converted.db', echo=False)

    # converting w data frame to sql function
    df.to_sql(file_name, con=engine, if_exists='replace', index=False)
    print(f"\nFile converted successfully to SQL table as {file_name}_converted.db!")
else:
    # protecting for errors
    print("\nError: Unsupported conversion type. Please choose 'csv', 'json', or 'sql'.")

What would you like to convert Historical_Fire_Data.csv to? (csv, json, sql): sql

File converted successfully to SQL table as Historical_Fire_Data_converted.db!
